In [1]:
import os
import shutil

In [2]:
base_image_path = 'brisc2025/segmentation_task/train/images'
nameImage = 'Sorted_Images'
folders_set = set()

bip = os.listdir(base_image_path)

for b in bip:
    remove_jpg =  b.replace(".jpg", "")
    split_b = remove_jpg.split('_')
    folder = '_'.join(split_b[3:6])
    folders_set.add(folder)
    
    if not os.path.isdir(nameImage):
        os.mkdir(nameImage)
        print(f"Folder '{nameImage}' created.") 

for folder in folders_set:
        sub = os.path.join(nameImage, folder)
        os.makedirs(sub, exist_ok=True)
        print(f"Folder '{sub}' created successfully.") 

In [3]:
base_mask_path = 'brisc2025/segmentation_task/train/masks'
nameMask = 'Sorted_Masks'
folders_set = set()

bip = os.listdir(base_mask_path)

for b in bip:
    remove_jpg =  b.replace(".jpg", "")
    split_b = remove_jpg.split('_')
    folder = '_'.join(split_b[3:6])
    folders_set.add(folder)
    print(folders_set)
    if not os.path.isdir(nameMask):
        os.mkdir(nameMask)
        print(f"Folder '{nameMask}' created.") 

for folder in folders_set:
        sub = os.path.join(nameMask, folder)
        os.makedirs(sub, exist_ok=True)
        print(f"Folder '{sub}' created successfully.") 

In [4]:
source_d = 'brisc2025/segmentation_task/train/masks' #change for basedir
dest_d = 'Sorted_Masks/pi_sa_t1' #change accordingly for subdir

for images in os.listdir(source_d):
    s_path = os.path.join(source_d, images)
    d_path = os.path.join(dest_d, images)

    if os.path.isfile(s_path):
        if images.__contains__('pi_sa_t1'): #chnage accordingly for subdir
            shutil.move(s_path, d_path)
            print(f"Moved '{images}' to '{d_path}'")

In [ ]:
sort_images = 'Sorted_Images'
sort_masks = 'Sorted_Masks'


for dirpath, dirname, files in os.walk(sort_images):
    for f1 in files:
        sort_image_path = os.path.join(dirpath, f1) 
        
for dirpath, dirname, files in os.walk(sort_masks):
    for f2 in files:
        sort_mask_path = os.path.join(dirpath, f2)
        split_smp = sort_mask_path.split('_')
        code_numberM = split_smp[5]
    
    for f in files:
        if f.__contains__(code_numberM):
                print(sort_mask_path)
                print(code_numberM)


In [6]:
numkeys = []

for dirpath, dirname, files in os.walk(sort_images):
    for f in files:
        jpg = f.replace(".jpg", "")
        split = jpg.split('_')
        numkey = split[2:6]
        joined_numkey = '_'.join(numkey)
        numkeys.append(joined_numkey) 

In [ ]:
#d_(x) <- dierectory 
#dn_(x) <- directory name
#f_(x) <- files in direcotry 

batch_paths = []

for (d_m, dn_ms, f_m), (d_i, dn_is, f_i) in zip(os.walk(sort_masks), os.walk(sort_images)):
        for f1, f2 in zip(f_i, f_m):
               full_path_i = os.path.join(d_i, f1)
               full_path_m = os.path.join(d_m, f2)
               
               for n in numkeys:
                       if full_path_i.__contains__(n) and full_path_m.__contains__(n):
                               batch_paths.append((full_path_i, full_path_m))

for batch in batch_paths:
    print(batch)

In [24]:
import torch
from torchvision import transforms 
from torch.utils.data import Dataset, DataLoader
from PIL import Image

In [25]:
print(torch.__version__)

2.8.0


In [ ]:

#IMG and Mask Transformations (resize 224, 224), tensor output
transform_img = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

transform_mask = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

#storage 
Tensor_Batch = []

for img_path, mask_path in batch_paths:
    
    image = Image.open(img_path).convert('L')
    mask = Image.open(mask_path).convert('L')

    tensorI = transform_img(image)
    tensorM = transform_mask(mask)
    Tensor_Batch.append((tensorI, tensorM))

print("size of dataset after processing:", len(Tensor_Batch))

In [10]:
#Custom Dataset

class ImageMask(Dataset):

    def __init__(self, batches, transform):
        self.data = batches 
        self.transform = transform

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
       image = self.data[idx]

       if self.transform:
           image = self.transform(image)

       return image

In [ ]:
custom_ds = ImageMask(Tensor_Batch, transform=True)

batch_size = 32
dl = DataLoader(Tensor_Batch, batch_size= batch_size, shuffle=False )

for batch in dl:
    print(type(batch)) # <class 'list>
    print(len(dl)) #amount per batch (3933/32 = 123)
    print(len(batch)) #2
    print("Dataset size:", len(dl.dataset)) #size of dataset
    break


In [37]:
import matplotlib.pyplot as plt

In [ ]:
#check that image and mask post tensor transform still line up
# (batch [0])

target_idx = 47 #batch entered 
target_pair = 10 #target pair

for batch, (images, masks) in enumerate (dl): #idx loop
    if batch == target_idx:
       
        img = images[target_pair].squeeze() # going from [1, 32, 224, 224] -> [224, 224]
        mask = masks[target_pair].squeeze() # going from [1, 32, 224, 224] -> [224, 224]
        
        # plotting stuff 
        fig, axs = plt.subplots(1, 2, figsize=(8, 4))

        axs[0].imshow(img, cmap="gray")
        axs[0].set_title("Image")
        axs[0].axis("off")

        axs[1].imshow(mask, cmap="gray")
        axs[1].set_title("Mask")
        axs[1].axis("off")

        plt.show()
        break

In [ ]:
# more plotting stuff to confrim img line up
fig, ax = plt.subplots(figsize=(3, 3))
ax.imshow(img, cmap="gray")
ax.imshow(mask, cmap="Reds", alpha=0.4)  # overlay mask in red
ax.set_title("Overlay")
ax.axis("off")
plt.show()